In [4]:
%reload_ext autoreload
%autoreload 2
import os,sys
sys.path.append('..')
sys.path.append('../ref/DropoutUncertaintyExps/net/') # Yarin Gal's keras implementation
import net as origin_imp
from BNN_Dropout import BNN_Dropout
import numpy as np
import torch
import torch.nn
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [31]:
 def get_data(dataset, split_id):
    ds_prefix = '../UCI_Datasets/'
    data = np.loadtxt('../UCI_Datasets/' + dataset + "/data/data.txt")
    xs = data[:,:-1]
    ys = data[:,-1]
    n_splits = np.loadtxt('../UCI_Datasets/' + dataset + "/data/n_splits.txt", dtype = np.int64)
    if split_id >= n_splits:
        train_x = None
        train_y = None
        test_x  = None
        test_y = None
        tau = None
        n_hidden = None
        n_epochs = None
    else:
        train_id = np.loadtxt('../UCI_Datasets/' + dataset + "/data/index_train_" + str(split_id) + ".txt", dtype = np.int64)
        test_id = np.loadtxt('../UCI_Datasets/' + dataset + "/data/index_test_" + str(split_id) + ".txt", dtype = np.int64)
        tau      = np.loadtxt('../UCI_Datasets/' + dataset + "/results/test_tau_100_xepochs_1_hidden_layers.txt")[split_id]
        n_hidden = np.loadtxt('../UCI_Datasets/' + dataset + "/data/n_hidden.txt", dtype = np.int64)
        n_epochs = np.loadtxt('../UCI_Datasets/' + dataset + "/data/n_epochs.txt", dtype = np.int64)

        train_x = xs[train_id]
        train_y = ys[train_id]
        test_x = xs[test_id]
        test_y = ys[test_id]
    return train_x, train_y, test_x, test_y,tau,n_hidden, n_splits, n_epochs

In [32]:
dataset = 'concrete'
split_id = 19


In [53]:
 def uci(dataset, split_id):
    train_x, train_y, test_x, test_y,tau, n_hiddens, n_splits, n_epochs = get_data(dataset, split_id)
    if(split_id >= n_splits):
        print("Invalid split_id")
        return np.nan, np.nan, np.nan, np.nan
    print('Dataset %s, split: %d' % (dataset, split_id))
    print('Model precision: %g' % tau)
    print('n_hiddens:%d' % n_hiddens)
    origin_model = origin_imp.net(train_x, train_y, n_hidden=[n_hiddens.item()], n_epochs = 100*n_epochs, normalize = True,tau = tau)
    _, rmse1, ll1 = origin_model.predict(test_x, test_y)
    print('RMSE = %g, NLL = %g' % (rmse1, -1 * ll1))

    conf = dict()
    conf['num_epochs'] = 100*n_epochs
    conf['normalize'] = True
    conf['tau'] = tau
    conf['lscale'] = 0.01
    conf['dropout_rate'] = 0.05
    conf['print_every'] = np.inf
    model = BNN_Dropout(train_x.shape[1], num_hiddens = [n_hiddens], conf = conf)
    model.train(torch.FloatTensor(train_x), torch.FloatTensor(train_y))
    rmse2,_,nll2 = model.validate(torch.FloatTensor(test_x), torch.FloatTensor(test_y), num_samples=1000)
    print('RMSE = %g, NLL = %g' % (rmse2, nll2))
    return rmse1, -1 * ll1, rmse2.item(), nll2.item()

In [54]:
ds = [
    'bostonHousing'
, 'concrete'
, 'energy'
, 'kin8nm'
, 'naval-propulsion-plant'
, 'power-plant'
, 'protein-tertiary-structure'
, 'wine-quality-red'
, 'yacht'

]

In [55]:
stat = dict()
for d in ds:
    stat[d] = []
    for split_id in range(20):
        stat[d].append(uci(d,split_id))
        
import pickle
f = open("stat.pkl","wb")
pickle.dump(stat,f)
f.close()
        

Dataset bostonHousing, split: 0
Model precision: 0.1
n_hiddens:50
51/51 [==============================] - 0s 3ms/step
RMSE = 2.1209, NLL = 2.33951
RMSE = 2.16741, NLL = 2.35009
Dataset bostonHousing, split: 1
Model precision: 0.1
n_hiddens:50
51/51 [==============================] - 0s 3ms/step
RMSE = 2.67449, NLL = 2.44612
RMSE = 2.62684, NLL = 2.43362
Dataset bostonHousing, split: 2
Model precision: 0.2
n_hiddens:50


KeyboardInterrupt: 